<a href="https://colab.research.google.com/github/Mohamm76/Arabic-Handwritten-Digits-Recognition-KSA/blob/main/notebook/Arabic_MNIST_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ربط الملف بجوجل درايف

In [ ]:
# 1. ربط Google Drive للوصول إلى المجلدات التي أنشأتها
from google.colab import drive
drive.mount('/content/drive')

# 2. تحديد المسار الرئيسي للمشروع (تأكد من مطابقة الاسم لما أنشأته في درايف)
import os
os.chdir('/content/drive/MyDrive/AI_Portfolio/Arabic_MNIST')
print("المسار الحالي هو:", os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
المسار الحالي هو: /content/drive/MyDrive/AI_Portfolio/Arabic_MNIST


# جلب البيانات وتخزينها
سنقوم الآن بتحميل مجموعة بيانات MADBase (الأرقام العربية) مباشرة إلى مجلد data في الدرايف الخاص بي لتجنب تكرار التحميل لاحقاً.

In [ ]:
# تحميل البيانات بصيغة CSV (الأسهل للمبتدئين) مباشرة إلى مجلد data
# سنستخدم نسخة GitHub الموثقة للبيانات لضمان سرعة التحميل
!wget -P data/ https://raw.githubusercontent.com/mloey/Arabic-Handwritten-Digits-Dataset/master/Arabic%20Handwritten%20Digits%20Dataset%20CSV.zip

# فك الضغط داخل مجلد data
import zipfile
with zipfile.ZipFile('data/Arabic Handwritten Digits Dataset CSV.zip', 'r') as zip_ref:
    zip_ref.extractall('data/')

print("تم تحميل البيانات وفك الضغط بنجاح في مجلد data")

--2026-01-28 09:03:27--  https://raw.githubusercontent.com/mloey/Arabic-Handwritten-Digits-Dataset/master/Arabic%20Handwritten%20Digits%20Dataset%20CSV.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13911478 (13M) [application/zip]
Saving to: ‘data/Arabic Handwritten Digits Dataset CSV.zip’

Arabic Handwritten  100%[===================>]  13.27M  33.1MB/s    in 0.4s    

2026-01-28 09:03:28 (33.1 MB/s) - ‘data/Arabic Handwritten Digits Dataset CSV.zip’ saved [13911478/13911478]

تم تحميل البيانات وفك الضغط بنجاح في مجلد data
